In [1]:
import ast
import numpy as np
import os
import pycountry
import pandas as pd
import re
import sqlite3

from io import StringIO
from sklearn.preprocessing import MinMaxScaler

In [2]:
def get_language_name(code):
    try:
        return pycountry.languages.get(alpha_2=code).name
    except:
        return 'Unknown'

In [3]:
def describe_data(df):
    output = StringIO()

    # get dimensions for dataframe
    num_rows, num_columns = df.shape
    print(f"Number of rows: {num_rows}", file=output)
    print(f"Number of columns: {num_columns}\n", file=output)

    # get missing values count and %
    print("Missing values per column:", file=output)
    missing_info = df.isnull().sum()
    for column, count in missing_info.items():
        percent = (count / num_rows) * 100
        print(f"  {column}: {count} missing ({percent:.2f}%)", file=output)
    print("\nColumn details:\n", file=output)

    # for each column check their type
    for column in df.columns:
        col_data = df[column]
        col_dtype = col_data.dtype
        print(f"Column: {column}, Type: {col_dtype}", file=output)

        # if numeric describe data and check it's skewness
        if pd.api.types.is_numeric_dtype(col_data):
            print(col_data.describe(), file=output)
            print(f"  Skewness: {col_data.skew():.2f}", file=output)

        # if date then display date range and no. of unique dates
        elif pd.api.types.is_datetime64_any_dtype(col_data):
            print(f"  Date Range: {col_data.min()} to {col_data.max()}", file=output)
            print(f"  Unique Dates: {col_data.nunique()}", file=output)

        # if categorical and  unique categories <= 10 then display categories
        elif isinstance(col_data.dtype, pd.CategoricalDtype) or col_data.dtype == 'object':
            num_categories = col_data.nunique()
            print(f"  Number of categories: {num_categories}", file=output)
            if num_categories <= 10:
                category_counts = col_data.value_counts()
                print("  Counts per category:", file=output)
                for cat, count in category_counts.items():
                    print(f"    {cat}: {count}", file=output)

        # else if unique counts <= 10 then display all of them
        else:
            unique_vals = col_data.unique()
            if len(unique_vals) <= 10:
                print("  Unique values:", file=output)
                for val in unique_vals:
                    print(f"    {val}", file=output)
        
        print("-" * 50, file=output)

    return output.getvalue()

In [4]:
# mean imputation for missing runtime
def mean_imputation(merged_df, col_name):
    merged_df[col_name] = merged_df.groupby('genre')[col_name].transform(
        lambda x: x.fillna(x.mean())
    )

    print("After genre wise mean imputation check if still there are any null values")
    print(merged_df[col_name].isnull().sum())

    print("performing overall mean imputation for remaining null values")
    merged_df[col_name] = merged_df[col_name].fillna(merged_df[col_name].mean())

    print(f"Verifying if all missing values are filled for column {col_name}")
    print(merged_df[col_name].isnull().sum())
    # print(merged_df[col_name].value_counts().to_dict())

    return merged_df

In [5]:
def normalizing_data_imdb(merged_df, col_name):
    col_name_id = col_name + "_id"
    col_name_name = col_name + "_name"
    col_name_db = col_name + "_imdb"
    # get genre_id and genre_name mapping for genre_schema
    unique_genres = merged_df[col_name].dropna().unique()
    genres_df = pd.DataFrame({
        col_name_id: range(1, len(unique_genres) + 1),
        col_name_name: unique_genres
    })

    # create mapping of above schema
    genre_map = dict(zip(genres_df[col_name_name], genres_df[col_name_id]))

    # get movie_id, genre_id mapping for schema storage
    movie_genre_map_df = merged_df[['movie_id', col_name]].copy()
    movie_genre_map_df[col_name_id] = movie_genre_map_df[col_name].map(genre_map)
    movie_genre_map_df = movie_genre_map_df[['movie_id', col_name_id]]

    # drop genre column as we have seprate schema for it now
    movies_df = merged_df.drop(columns=[col_name])  # Drop genre column to normalize
    movies_df = movies_df.drop_duplicates(subset=['movie_id'])  # Optional: avoid duplicates

    # write all the schemas to DB
    # connect to the SQLite database
    db_name = '../Data/movies.db'
    conn = sqlite3.connect(db_name)

    # insert genre data to output table
    genres_df.to_sql(col_name, conn, if_exists='replace', index=False)
    movie_genre_map_df.to_sql(col_name_db, conn, if_exists='replace', index=False)

    conn.commit()
    conn.close()
    
    return movies_df

In [6]:
def normalizing_data_tmdb(merged_df, col_name):
    col_name_id = col_name + "_id"
    col_name_name = col_name + "_name"
    col_name_db = col_name + "_tmdb"
    # get genre_id and genre_name mapping for genre_schema
    unique_genres = merged_df[col_name].dropna().unique()
    genres_df = pd.DataFrame({
        col_name_id: range(1, len(unique_genres) + 1),
        col_name_name: unique_genres
    })

    # create mapping of above schema
    genre_map = dict(zip(genres_df[col_name_name], genres_df[col_name_id]))

    # get movie_id, genre_id mapping for schema storage
    movie_genre_map_df = merged_df[['id', col_name]].copy()
    movie_genre_map_df[col_name_id] = movie_genre_map_df[col_name].map(genre_map)
    movie_genre_map_df = movie_genre_map_df[['id', col_name_id]]

    # drop genre column as we have seprate schema for it now
    movies_df = merged_df.drop(columns=[col_name])  # Drop genre column to normalize
    movies_df = movies_df.drop_duplicates(subset=['id'])  # Optional: avoid duplicates

    # write all the schemas to DB
    # connect to the SQLite database
    db_name = '../Data/movies.db'
    conn = sqlite3.connect(db_name)

    # insert genre data to output table
    genres_df.to_sql(col_name, conn, if_exists='replace', index=False)
    movie_genre_map_df.to_sql(col_name_db, conn, if_exists='replace', index=False)

    conn.commit()
    conn.close()
    
    return movies_df

In [7]:
def imdb():
    directory_path = '../Data/IMDB'

    print("\n***** All IMDB files are separated by genre, but contains same columns *****")
    print("\n***** So let's merge them, for easy processing. Note: they do have 'genre' column *****")
    
    # List all files and directories in the current directory
    files_in_directory = os.listdir(directory_path)
    df_list = []
    
    for filename in files_in_directory:
        if filename.endswith(".csv"):
            df_list.append(pd.read_csv(f"{directory_path}/{filename}"))
            
    merged_df = pd.concat(df_list, ignore_index = True)
    print("\n***** Merged dataset dimensions are: *****")
    
    total_rows, total_cols = merged_df.shape
    print(f"Number of rows {total_rows} and Number of cols {total_cols}")

    # print("\n***** Display Top5 records *****")
    # print(merged_df.head())

    print("\n***** Describing Data *****")
    print(describe_data(merged_df))

    print("\n***** Dropping the columns 'certificate' and 'gross(is $)' due to high missing data. Also, 'director_id' and 'star_id', as we will create new id mapping for them *****")
    # note that certificate and gross(in $) has more than 70% missing values
    # so they are not useful for us, hence drop them
    merged_df.drop(columns = ["certificate", "gross(in $)", "director_id", "star_id"], inplace = True)
    #print(merged_df.shape)

    print("\n***** Dropping the 4 records which does not have movie name *****")
    # drop movie records which does not have name
    merged_df = merged_df.dropna(subset=['movie_name'])
    # print(merged_df['movie_name'].isna().sum())

    # converting movie_id to int
    merged_df['movie_id'] = merged_df['movie_id'].str[2:].astype('Int64')

    print("\n***** Keeping data only for year >= 2000 *****")
    merged_df['year'] = pd.to_numeric(merged_df['year'], errors='coerce')
    # merged_df = merged_df[merged_df['year'] >= 2000]

    print(describe_data(merged_df))

    print("\n***** Format 'runtime' column to have only numerical (float) values *****")
    # process the runtime attribute and bring to int format
    # Define regex pattern for valid entries like '130 min'
    pattern = r'^\d+\s*min$'
    
    # Create a mask for rows that do NOT match the pattern 
    # (e.g. to check if it contains 2 hrs or 2.5 hrs etc)
    invalid_runtime = merged_df[~merged_df['runtime'].astype(str).str.match(pattern, na=False)]
    
    # Display the rows with invalid format
    print("Invalid runtime entries:")
    print(invalid_runtime['runtime'].value_counts())
    
    # Remove commas and extract numeric part
    merged_df['runtime'] = merged_df['runtime'].str.replace(',', '')  # Remove commas
    merged_df['runtime'] = merged_df['runtime'].str.extract(r'(\d+)').astype(float)  # Use raw string
    # print(merged_df['runtime'].head())

    print("\n***** Genre column is list type, exploding it to perform imputations on 'rating', 'vote' and col_name columns *****")

    # note: genre column is a string where it contains multiple genres separated by commas
    # split it and then explode
    merged_df['genre'] = merged_df['genre'].apply(
        lambda x: [g.strip() for g in x.split(',')] if pd.notna(x) else x
    )

    # check shape before explode
    # print(merged_df.shape)
    # Step 2: Explode
    merged_df = merged_df.explode('genre')
    # check shape after explode
    # print(merged_df.shape)

    # after exploding there are duplicates
    # e.g. movie A can have both genre "Action" and "Comedy"
    # then it will have it's seperate record in both of the files (which we merged initially)
    merged_df.drop_duplicates(inplace = True)
    # print(merged_df.shape)
    # print(merged_df.columns)

    # mean imputation for missing runtime
    merged_df = mean_imputation(merged_df, 'runtime')
    merged_df['runtime'] = merged_df['runtime'].round()

    # mean imputation for missing votes
    merged_df = mean_imputation(merged_df, 'votes')
    merged_df['votes'] = merged_df['votes'].round()

    # mean imputation for missing rating
    merged_df = mean_imputation(merged_df, 'rating')
    merged_df['rating'] = merged_df['rating'].round(1)

    print("\n ***** Now let's analyse again how our processed data looks like *****")
    print(describe_data(merged_df))

    print("\n ***** Actors and directors column still have some missing values, but we won't impute them *****")
    print("\n ***** Instead we will do normalization on data *****")

    print("\n ***** Starting Normalization for 'genre', 'actor' and 'diector' columns *****")

    merged_df = normalizing_data_imdb(merged_df, 'genre')

    # fill missing values for director
    merged_df['director'] = merged_df['director'].fillna('Unknown')
    merged_df['director'] = merged_df['director'].str.replace('\n', '').str.strip()

    # note: genre column is a string where it contains multiple genres separated by commas
    # split it and then explode
    merged_df['director'] = merged_df['director'].apply(
        lambda x: [g.strip() for g in x.split(',')] if pd.notna(x) else x
    )

    # check shape before explode
    # print(merged_df.shape)
    # Step 2: Explode
    merged_df = merged_df.explode('director')
    # check shape after explode
    # print(merged_df.shape)

    # after exploding there are duplicates
    # e.g. as some values can have different order [director_A, director_B] and [director_B, director_A]
    # then after splitting it will create duplicates
    merged_df.drop_duplicates(inplace = True)
    # print(merged_df.shape)
    # print(merged_df.columns)

    merged_df = normalizing_data_imdb(merged_df, 'director')

    # fill missing values for star
    merged_df['star'] = merged_df['star'].fillna('Unknown')
    merged_df['star'] = merged_df['star'].str.replace('\n', '').str.strip()

    # note: genre column is a string where it contains multiple genres separated by commas
    # split it and then explode
    merged_df['star'] = merged_df['star'].apply(
        lambda x: [g.strip() for g in x.split(',')] if pd.notna(x) else x
    )

    # check shape before explode
    # print(merged_df.shape)
    # Step 2: Explode
    merged_df = merged_df.explode('star')
    # check shape after explode
    # print(merged_df.shape)

    # after exploding there are duplicates
    # e.g. as some values can have different order [director_A, director_B] and [director_B, director_A]
    # then after splitting it will create duplicates
    merged_df.drop_duplicates(inplace = True)
    # print(merged_df.shape)
    # print(merged_df.columns)

    movies_df = normalizing_data_imdb(merged_df, 'star')

    # converting data types for year and runtime to 'Int'
    movies_df['year'] = movies_df['year'].astype('Int64')
    movies_df['runtime'] = movies_df['runtime'].astype('Int64')

    print(describe_data(merged_df))
    # write all the schemas to DB
    # connect to the SQLite database
    db_name = '../Data/movies.db'
    conn = sqlite3.connect(db_name)

    # insert movie data to output table
    movies_df.to_sql('imdb', conn, if_exists='replace', index=False)

    conn.commit()
    conn.close()
    

In [8]:
imdb()


***** All IMDB files are separated by genre, but contains same columns *****

***** So let's merge them, for easy processing. Note: they do have 'genre' column *****

***** Merged dataset dimensions are: *****
Number of rows 368300 and Number of cols 14

***** Describing Data *****
Number of rows: 368300
Number of columns: 14

Missing values per column:
  movie_id: 0 missing (0.00%)
  movie_name: 4 missing (0.00%)
  year: 53248 missing (14.46%)
  certificate: 264109 missing (71.71%)
  runtime: 109154 missing (29.64%)
  genre: 0 missing (0.00%)
  rating: 137362 missing (37.30%)
  description: 0 missing (0.00%)
  director: 27369 missing (7.43%)
  director_id: 27369 missing (7.43%)
  star: 58695 missing (15.94%)
  star_id: 51858 missing (14.08%)
  votes: 137358 missing (37.30%)
  gross(in $): 343261 missing (93.20%)

Column details:

Column: movie_id, Type: object
  Number of categories: 243197
--------------------------------------------------
Column: movie_name, Type: object
  Number o

In [11]:
def tmdb():
    directory_path = '../Data/TMDB'

    # List all files and directories in the current directory
    files_in_directory = os.listdir(directory_path)
    df_list = []
    
    for filename in files_in_directory:
        if filename.endswith(".csv"):
            df_list.append(pd.read_csv(f"{directory_path}/{filename}"))
            
    merged_df = pd.concat(df_list, ignore_index = True)
    print("\n***** Merged dataset dimensions are: *****")
    
    total_rows, total_cols = merged_df.shape
    print(f"Number of rows {total_rows} and Number of cols {total_cols}")

    print("\n***** Display Top5 records *****")
    print(merged_df.head())

    print("\n***** Describing Data *****")
    print(describe_data(merged_df))

    print("\n***** Dropping the columns 'backdrop_path', 'homepage', 'imdb_id', 'overview', 'poster_path', 'tagline', 'genres', 'keywords' due to high missing data.  *****")
    # note that certificate and gross(in $) has more than 70% missing values
    # so they are not useful for us, hence drop them
    merged_df.drop(columns = ["backdrop_path", "homepage", "imdb_id", "overview", "poster_path", "tagline", "genres", "keywords"], inplace = True)
    #print(merged_df.shape)

    print("\n***** Dropping the 13 records which does not have title *****")
    # drop movie records which does not have name
    merged_df = merged_df.dropna(subset=['title'])
    # print(merged_df['movie_name'].isna().sum())

    # converting movie_id to int
    merged_df['id'] = merged_df['id'].astype('Int64')

    print("\n ***** Filling missing values for columns 'production_countries', 'production_companies' and 'spoken_language' *****")
    merged_df['production_countries'] = merged_df['production_countries'].fillna('Unknown')
    merged_df['production_companies'] = merged_df['production_companies'].fillna('Unknown')
    merged_df['spoken_languages'] = merged_df['spoken_languages'].fillna('Unknown')

    print("\n ***** Converting release date to release year *****")
    merged_df['release_date'] = pd.to_datetime(merged_df['release_date'], errors='coerce')

    merged_df['release_year'] = merged_df['release_date'].dt.year
    merged_df.drop(columns=['release_date'], inplace = True)

    print("\n ***** Starting Normalization on columns 'production_countries', 'production_companies', 'spoken_languages' ***** ")

    merged_df['production_companies'] = merged_df['production_companies'].apply(
        lambda x: [g.strip() for g in x.split(',')] if pd.notna(x) else x
    )

    # check shape before explode
    # print(merged_df.shape)
    # Step 2: Explode
    merged_df = merged_df.explode('production_companies')
    # check shape after explode
    # print(merged_df.shape)

    merged_df.drop_duplicates(inplace = True)
    # print(merged_df.shape)
    # print(merged_df.columns)

    merged_df = normalizing_data_tmdb(merged_df, 'production_companies')

    # starting for production_countries
    merged_df['production_countries'] = merged_df['production_countries'].apply(
        lambda x: [g.strip() for g in x.split(',')] if pd.notna(x) else x
    )

    # check shape before explode
    # print(merged_df.shape)
    # Step 2: Explode
    merged_df = merged_df.explode('production_countries')
    # check shape after explode
    # print(merged_df.shape)

    merged_df.drop_duplicates(inplace = True)
    # print(merged_df.shape)
    # print(merged_df.columns)

    merged_df = normalizing_data_tmdb(merged_df, 'production_countries')

    print(describe_data(merged_df))

    # starting for spoken_languages
    merged_df['spoken_languages'] = merged_df['spoken_languages'].apply(
        lambda x: [g.strip() for g in x.split(',')] if pd.notna(x) else x
    )

    # check shape before explode
    # print(merged_df.shape)
    # Step 2: Explode
    merged_df = merged_df.explode('spoken_languages')
    # check shape after explode
    # print(merged_df.shape)

    merged_df.drop_duplicates(inplace = True)
    # print(merged_df.shape)
    # print(merged_df.columns)

    movies_df = normalizing_data_tmdb(merged_df, 'spoken_languages')

    # converting language codes to actual language names
    movies_df['original_language'] = movies_df['original_language'].apply(get_language_name)

    print("\n***** Removing noisy data from runtime column *****")
    movies_df['runtime'] = movies_df['runtime'].apply(lambda x: 0 if x < 0 else x)

    print(describe_data(movies_df))

    # rounding up vote_avg to 1 decimal point
    movies_df['vote_average'] = movies_df['vote_average'].round(1)

    # write all the schemas to DB
    # connect to the SQLite database
    db_name = '../Data/movies.db'
    conn = sqlite3.connect(db_name)

    # insert movie data to output table
    movies_df.to_sql('tmdb', conn, if_exists='replace', index=False)

    conn.commit()
    conn.close()
    

In [12]:
tmdb()


***** Merged dataset dimensions are: *****
Number of rows 1180936 and Number of cols 24

***** Display Top5 records *****
       id            title  vote_average  vote_count    status release_date  \
0   27205        Inception         8.364       34495  Released   2010-07-15   
1  157336     Interstellar         8.417       32571  Released   2014-11-05   
2     155  The Dark Knight         8.512       30619  Released   2008-07-16   
3   19995           Avatar         7.573       29815  Released   2009-12-15   
4   24428     The Avengers         7.710       29166  Released   2012-04-25   

      revenue  runtime  adult                     backdrop_path  ...  \
0   825532764      148  False  /8ZTVqvKDQ8emSGUEMjsS4yHAwrp.jpg  ...   
1   701729206      169  False  /pbrkL804c8yAv3zBZR4QPEafpAR.jpg  ...   
2  1004558444      152  False  /nMKdUUepR0i5zn0y1T4CsSB5chy.jpg  ...   
3  2923706026      162  False  /vL5LR6WdxWPjLPFRLe133jXWsh5.jpg  ...   
4  1518815515      143  False  /9BBTo63ANS

In [13]:
def movielens():
    directory_path = '../Data/movielens'

    # List all files and directories in the current directory
    files_in_directory = os.listdir(directory_path)
    df_list = []

    db_name = '../Data/movies.db'
    conn = sqlite3.connect(db_name)
    
    for filename in files_in_directory:
        if filename.endswith(".csv"):
            df = pd.read_csv(f"{directory_path}/{filename}")

            print(filename)
            df.dropna(inplace = True)

            print(describe_data(df))

            table_name = filename.split(".")[0]
        
            # insert movie data to output table
            df.to_sql(table_name, conn, if_exists='replace', index=False)
        
    conn.commit()
    conn.close()


In [14]:
movielens()

links.csv
Number of rows: 87461
Number of columns: 3

Missing values per column:
  movieId: 0 missing (0.00%)
  imdbId: 0 missing (0.00%)
  tmdbId: 0 missing (0.00%)

Column details:

Column: movieId, Type: int64
count     87461.000000
mean     157739.966682
std       78995.211045
min           1.000000
25%      112913.000000
50%      165835.000000
75%      213249.000000
max      292757.000000
Name: movieId, dtype: float64
  Skewness: -0.39
--------------------------------------------------
Column: imdbId, Type: int64
count    8.746100e+04
mean     2.794260e+06
std      4.280858e+06
min      1.000000e+00
25%      9.459000e+04
50%      4.928900e+05
75%      3.881680e+06
max      2.908110e+07
Name: imdbId, dtype: float64
  Skewness: 2.24
--------------------------------------------------
Column: tmdbId, Type: float64
count    8.746100e+04
mean     2.413823e+05
std      2.471467e+05
min      2.000000e+00
25%      4.683600e+04
50%      1.392720e+05
75%      3.816930e+05
max      1.186337e+